# Read Me

This project uses linkedin to send automatic messages in order to find a job faster

### Goals of the project
- Finding recruiters and people working in the industry i want and send them appropriate messages (adding the possibility to schedule a meeting with me on my personnal calendar ?
- Apply to the positions that fits my criteria (only working for simplified job apply imo)


### TODO
- Find optimal recruiter, find out how the localization filter works
- Work on finding optimal job post to send message to the recruiter from the job post


### pipeline to dm recruiters from job post
- get list of job throught the job search bar from linkedin (still can optimize this throught better keywords and location not working atm)
- iterate throught each job, if there is a recruiter who writed the job post, get it's id from selenium scrap
- send a connection invite proposing to have a quick chat about the role (add few keywords ?)
- connect the response to a collab excel to save the info


In [1]:
from linkedin_api import Linkedin
from linkedin_api.utils.helpers import get_id_from_urn

In [3]:
# Authenticate using any Linkedin account credentials


test_credential = get_credential('test_credential.txt')
true_credential = get_credential('linkedin_username.txt')

test_api = Linkedin(test_credential[0], test_credential[0])
real_api = Linkedin(true_credential[0], true_credential[0])


# GET a profile
#profile = api.get_profile('billy-g')

# GET a profiles contact info
#contact_info = api.get_profile_contact_info('billy-g')

# GET 1st degree connections of a given profile    USE URN ID
#connections = api.get_profile_connections('1234asc12304')


# First part : finding recruiters and sending them invite with personnal messages

- First dichotomy, from their summary we try to define if they are in the quant finance or in the tech sector to send the more accurate message

- Second one, if they are based in France we send our message in French

- To get better results we could developp a rating on how much we would be interested in each profile

In [4]:
#message_data = "I'm a graduate from an MSc in Statistics from Université PSL in Paris, " \
#    + "after a year of experience as a Data Scientist, I am looking for" \
#    + "a new opportunity in this sector."   

#message_quant = "I'm a graduate from an MSc in Statistics and Quantitative Finance from Université " \
#    + "PSL in Paris, after a year of experience as a Data Scientist, I am now looking to" \
#    + "work as a Quant."  


message_quant = "graduate from an MSc in Statistics and Quantitative Finance. An experience as a data scientist, looking for an opportunity as a quant"
message_data = "graduate from an MSc in Statistics. An experience as a data scientist, looking for a new opportunity, data-driven."
    
end_of_message = "Feel free to contact me if you see a possible fit !"


data_recruiter = 0
quant_recruiter = 0
na_recruiter = 0

connections = []


regions_known = {'USA' : '103644278', 'France' : '105015875', 'UK' : '101165590', 'Holand' : '102890719'}


In [6]:
def create_message_for_recruiter_from_profile(recruiter_profile_id, api = test_api):
    
    recruiter = api.get_profile(recruiter_profile_id)
    first_name = recruiter['firstName']
    
    if(first_name):
        
        message = "Hello "+ first_name + ", \n" + "I am a graduate from an Msc in Statistics with an experience " \
                  + "as a Data Scientist and I came across your profile. I am currently looking for a new position." \
                  + " Would you be available for an exchange to see if you might have an opportunity that could " \
                  + "fit with me ?"

    else:
        
        message = "Hello, I am a graduate from an Msc in Statistics with an experience" \
                  + "as a Data Scientist and I came across your job post for a " + job_title \
                  + " role. I am very interested in this opportunity and would love to learn more about the position."

    
    return message

In [10]:
test_api.graphql_search_people(job_title = 'recruiter', regions = list(regions_known.values()), offset = 0, limit = 10)

[{'summary': "URBAN LINKER est avant tout un concept novateur de cabinet de recrutement et chasse de tête, spécialisé dans les métiers du digital, implanté en France à Paris, Lyon et Bordeaux.\n\nAu-delà du recrutement, nos activités de conseil, l'organisation de meetups, l'hébergement des événements Web les plus courus du moment ainsi que les nombreux partenariats que nous avons noué font d'Urban Linker un acteur majeur de l'écosystème digital & start-up.\n\nAujourd’hui, Urban Linker, c’est :\n\n• Une start-up composée de spécialistes et de passionnés de l’univers du digital\n• Un process de recrutement sur-mesure pour les acteurs du Web\n• Une société qui parle le même langage que les candidats qu’elle recrute\n• Un rôle de conseil sur des projets à forte croissance\n• Des locaux en plein Lyon situés rue Mercière\n\n\nSpécialités :\nNotre concept est innovant et intègre une expertise sur l'ensemble des métiers du digital.\n\n> TECHNIQUE : développeur Web (Php, Java, .Net…), front-end

In [7]:
search_quant = test_api.search_people(keywords = 'quantitative finance', keyword_title = 'recruiter', regions = list(regions_known.values()), limit = 100)

In [31]:
people_searched_tech = real_api.search_people(keywords = 'AI', keyword_title = 'recruiter', regions = list(regions_known.values()), limit = 200)

In [14]:
# Create sheet to see message send to recruiter from people search

mail = "tang.bellec@gmail.com"
#sheet = create_sheet("messages_recruiter_people_search", mail)
sheet = connect_to_sheet("messages_recruiter_people_search")
#add_new_row(sheet, ["recruiter_id", "message", "date", "answer"])

In [9]:
def extract_only_dict_from_gpt_output(gpt_answer):
    in_json = False
    
    for current_char in gpt_answer:
        if(in_json):
            json = json + current_char
            if(current_char == '}'):
                return json
        else:
            if(current_char == '{'):
                json = '{'
                in_json = True

In [108]:
# Pipeline without gpt for quant jobs

complete_message_quant ="I recently graduated with an MSc in Statistics and Quantitative Finance from "\
    + "Université PSL. As a passionate individual with an experience as a Data Scientist, I'm actively "\
    + "seeking a Quant position. I would love to discuss potential opportunities with you. "

connections = []
quant_recruiter = 0
data_recruiter = 0

from datetime import datetime
date_today = datetime.today().strftime('%Y-%m-%d')

for recruiter in search_quant[20:50]:
        
    first_name = recruiter['name'].split(" ")[0]
    final_message = "Hello " + first_name + " " + complete_message_quant
    real_api.add_connection(recruiter['public_id'], final_message)
    add_new_row(sheet, [recruiter['public_id'], current_connection[2], date_today, ""])

In [144]:
# Pipeline without gpt for tech jobs

complete_message_ds = "I recently graduated with an MSc in Statistics from "\
    + "Université Paris Dauphine. As a data-driven individual with an experience as a Data Scientist, I'm actively "\
    + "seeking a Data Scientist / ML Engineer position. I would love to discuss potential opportunities with you."


complete_message_ds = "I recently graduated with an MSc in Statistics from "\
    + "Université Paris Dauphine. As a data-driven individual with an experience as a Data Scientist, "\
    + "I believe you may have opportunities that would fit my profile. Looking forward to discussing" \
    + "any open positions. Thanks!"

In [110]:
# Pipeline with my real account
# https://www.linkedin.com/mynetwork/invitation-manager/sent/ 

connections = []
quant_recruiter = 0
data_recruiter = 0

from datetime import datetime
date_today = datetime.today().strftime('%Y-%m-%d')

for recruiter in people_searched_tech[101:]:
        
    try:
        current_connection = add_gpt_personnalized_connection(recruiter, real_api)
        add_new_row(sheet, [recruiter['public_id'], current_connection[2], date_today, ""])
        print('no key error')
        
    except KeyError:    # Happends when no summary or profile can't be found
        print('key error')
        continue


Brian Crewe
Here's the JSON output:

```
{
    "message": "Hello Brian, with my qualifications and experience in data science and analytics, I believe there could be opportunities in your client portfolio. I'm interested in learning more about possible opportunities that match my profile."
}
```
{
    "message": "Hello Brian, with my qualifications and experience in data science and analytics, I believe there could be opportunities in your client portfolio. I'm interested in learning more about possible opportunities that match my profile."
}
Hello Brian, with my qualifications and experience in data science and analytics, I believe there could be opportunities in your client portfolio. I'm interested in learning more about possible opportunities that match my profile.
no key error
Alina Constantin
Here is the JSON format message:

```
{
  "message": "Hi Alina, with my experience as a data scientist, advanced skills in boolean searching and stakeholder management, as well as my MSc in 

no key error
Jessica Castellanos
Here's the JSON message as requested:

```json
{
  "message": "Hi Jessica, hope you're doing great. Based on my experience and qualification, I believe there might be opportunities that would fit my profile. Would be glad to know more. Thanks!"
}
```
{
  "message": "Hi Jessica, hope you're doing great. Based on my experience and qualification, I believe there might be opportunities that would fit my profile. Would be glad to know more. Thanks!"
}
Hi Jessica, hope you're doing great. Based on my experience and qualification, I believe there might be opportunities that would fit my profile. Would be glad to know more. Thanks!
no key error
Rehan Syed
Here is the requested JSON message:

```
{
   "message": "Hi Rehan, with a MSc in Statistics and experience in data science, I believe my skills and experience fit perfectly with your core specialisation in data engineering and data science. Please let me know if there are any opportunities that match my profi

JSONDecodeError: Expecting property name enclosed in double quotes: line 3 column 1 (char 213)

## Work to send follow up messages after getting a new connection

In [15]:
conv = real_api.get_conversations()

In [10]:
def send_follow_up_message(api, base_message, follow_up_message, limit = 100):
    
    conversations = api.get_conversations()
    
    for counter_message in range(min(limit, len(conversations['elements']))):
        
        last_message = conversations['elements'][counter_message]['events'][0]['eventContent']['com.linkedin.voyager.messaging.event.MessageEvent']['attributedBody']['text']
                
        if(base_message in last_message):
            
            conv_id = conversations['elements'][counter_message]['dashEntityUrn'].split(':')[-1]   
            api.send_message(follow_up_message, conv_id)
            
    return True


In [17]:
conversations = real_api.get_conversations()

In [18]:
len(conversations['elements'])

20

In [20]:
conv_test = test_api.get_conversation_details('ACoAAAoJMUcBVFSXLQVBgpB7-fazv42w-EhaE7c')

In [35]:
conv_test.keys()

dict_keys(['dashEntityUrn', 'inboxType', 'notificationStatus', 'read', 'groupChat', 'totalEventCount', 'unreadCount', 'lastActivityAt', 'firstMessageUrn', 'backendUrn', 'receipts', 'archived', 'blocked', 'starred', 'entityUrn', 'sdkEntityUrn', 'viewerCurrentParticipant', 'featureTypes', 'withNonConnection', 'muted', 'events', 'participants', 'id'])

In [198]:
conversations['elements'][19]

{'dashEntityUrn': 'urn:li:fsd_conversation:2-MGQ3YzlkNmYtNDcwYi00MTEyLWE4MTgtNmQ0OTUxNmNhN2U0XzAxMg==',
 'inboxType': 'PRIMARY',
 'notificationStatus': 'ACTIVE',
 'read': True,
 'groupChat': False,
 'totalEventCount': 1,
 'unreadCount': 0,
 'lastActivityAt': 1684318252454,
 'firstMessageUrn': 'urn:li:fs_event:(2-MGQ3YzlkNmYtNDcwYi00MTEyLWE4MTgtNmQ0OTUxNmNhN2U0XzAxMg==,2-MTY4NDMxNzU3MTQ1OWIyMDkyNi0wMDMmMGQ3YzlkNmYtNDcwYi00MTEyLWE4MTgtNmQ0OTUxNmNhN2U0XzAxMg==)',
 'backendUrn': 'urn:li:messagingThread:2-MGQ3YzlkNmYtNDcwYi00MTEyLWE4MTgtNmQ0OTUxNmNhN2U0XzAxMg==',
 'receipts': [],
 'lastReadAt': 1684318099747,
 'archived': False,
 'blocked': False,
 'starred': False,
 'entityUrn': 'urn:li:fs_conversation:2-MGQ3YzlkNmYtNDcwYi00MTEyLWE4MTgtNmQ0OTUxNmNhN2U0XzAxMg==',
 'sdkEntityUrn': 'urn:li:msg_conversation:(urn:li:fsd_profile:ACoAADRK5hYBGMJvZMHyTclwGIHUeJVuGK-eN_A,2-MGQ3YzlkNmYtNDcwYi00MTEyLWE4MTgtNmQ0OTUxNmNhN2U0XzAxMg==)',
 'viewerCurrentParticipant': True,
 'featureTypes': ['REACTIONS',
 

In [191]:
base_message = 'MSc'
follow_up_message = 'Thank you for connecting with me! \n I was wondering if you have a moment for a brief call. '\
                    + 'This would provide me with the chance to introduce myself and discuss the specific '\
                    + 'roles you are recruiting for. '\
                    + 'Additionally, it would allow us to explore whether there could be a potential match.'\
                    + '\nBest regards, \n Tanguy Bellec'

send_follow_up_message(real_api, base_message, follow_up_message, 150)

Hi Daniél, with my MSc in Statistics and experience as a data scientist, I believe you might have job opportunities that fit my qualifications. Looking forward to discussing further opportunities with you.
2-ODM0Y2E5YzktZjkzNy00NDk5LTk2NmQtMDMxNTQzYTU4MzU2XzAxMw==
Hello Tamryn I recently graduated with an MSc in Statistics and Quantitative Finance from Université PSL. As a passionate individual with an experience as a Data Scientist, I'm actively seeking a Quant position. I would love to discuss potential opportunities with you. 
2-MGMyNTY4ZWMtZTM0ZS00NjVlLWJiMjQtNWUwZWQzYmUzYTg1XzAxMg==
Hello David I recently graduated with an MSc in Statistics and Quantitative Finance from Université PSL. As a passionate individual with an experience as a Data Scientist, I'm actively seeking a Quant position. I would love to discuss potential opportunities with you. 
2-ZmQ3YmNlMDQtNzRhNC00NmUxLWExMWItOTIwMzAxMGRhZTdjXzAxMg==
Hello Zoe I recently graduated with an MSc in Statistics and Quantitative Fi

True

In [23]:
t = test_api.get_profile('monikamimrova')

In [24]:
t['profile_urn'].split(':')[-1]

'ACoAAAoJMUcBVFSXLQVBgpB7-fazv42w-EhaE7c'

In [25]:
conv['elements'][10]['events'][0]['eventContent']['com.linkedin.voyager.messaging.event.MessageEvent']['attributedBody']['text']

'Thank you for connecting with me! \n I was wondering if you have a moment for a brief call. This would provide me with the chance to introduce myself and discuss the specific roles you are recruiting for. Additionally, it would allow us to explore whether there could be a potential match.\nBest regards, \n Tanguy Bellec'

In [26]:
t = 'I believe there may be opportunities that fit my profile'
t2 = conv['elements'][4]['events'][0]['eventContent']['com.linkedin.voyager.messaging.event.MessageEvent']['attributedBody']['text']

if t in t2:
    print('yes')

In [27]:
t2

'Thank you for connecting with me! \n I was wondering if you have a moment for a brief call. This would provide me with the chance to introduce myself and discuss the specific roles you are recruiting for. Additionally, it would allow us to explore whether there could be a potential match.\nBest regards, \n Tanguy Bellec'

In [28]:
len(conv['elements'])

20

In [149]:
conv['elements'][1]

{'dashEntityUrn': 'urn:li:fsd_conversation:2-ZTRlYzYyYjUtOWM5OC00NmM3LTljMDYtZDAzMDIwNDk4ZjNmXzAxMA==',
 'inboxType': 'PRIMARY',
 'notificationStatus': 'ACTIVE',
 'read': True,
 'groupChat': False,
 'totalEventCount': 1,
 'unreadCount': 0,
 'lastActivityAt': 1684831875714,
 'firstMessageUrn': 'urn:li:fs_event:(2-ZTRlYzYyYjUtOWM5OC00NmM3LTljMDYtZDAzMDIwNDk4ZjNmXzAxMA==,2-MTY4NDQyNTgzNTI0MmIxMzQ0MS0wMDEmZTRlYzYyYjUtOWM5OC00NmM3LTljMDYtZDAzMDIwNDk4ZjNmXzAxMA==)',
 'backendUrn': 'urn:li:messagingThread:2-ZTRlYzYyYjUtOWM5OC00NmM3LTljMDYtZDAzMDIwNDk4ZjNmXzAxMA==',
 'receipts': [],
 'lastReadAt': 1684426554884,
 'archived': False,
 'blocked': False,
 'starred': False,
 'entityUrn': 'urn:li:fs_conversation:2-ZTRlYzYyYjUtOWM5OC00NmM3LTljMDYtZDAzMDIwNDk4ZjNmXzAxMA==',
 'sdkEntityUrn': 'urn:li:msg_conversation:(urn:li:fsd_profile:ACoAADRK5hYBGMJvZMHyTclwGIHUeJVuGK-eN_A,2-ZTRlYzYyYjUtOWM5OC00NmM3LTljMDYtZDAzMDIwNDk4ZjNmXzAxMA==)',
 'viewerCurrentParticipant': True,
 'featureTypes': ['REACTIONS',
 

In [147]:
conv['elements'][0]['totalEventCount']

2

In [130]:
real_api.get_conversation('urn:li:fsd_conversation:2-OTE4YzI3YTYtZmY1NC00MTkyLWE5NzAtNzM0MmM2YzY5NGJjXzAxMA==')

{'status': 422}

In [135]:
conv['elements'][0]['events'][0]['eventContent']

{'com.linkedin.voyager.messaging.event.MessageEvent': {'messageBodyRenderFormat': 'DEFAULT',
  'body': '',
  'attributedBody': {'text': 'Hello Rea,\n\nI hope this message finds you well. I was wondering if you have a moment for a brief call. This would provide me with the chance to introduce myself and discuss the specific roles you are currently recruiting for. Additionally, it would allow us to explore whether there could be a potential match.\n\nBest regards,\nTanguy'}}}

In [137]:
conv['elements'][0]['events'][0]['eventContent']['com.linkedin.voyager.messaging.event.MessageEvent']['attributedBody']['text']

'Hello Rea,\n\nI hope this message finds you well. I was wondering if you have a moment for a brief call. This would provide me with the chance to introduce myself and discuss the specific roles you are currently recruiting for. Additionally, it would allow us to explore whether there could be a potential match.\n\nBest regards,\nTanguy'

In [131]:
conv['elements'][0]['events'][0].keys()
#['events'][0]

dict_keys(['createdAt', 'reactionSummaries', 'dashEntityUrn', 'quickReplyRecommendations', 'entityUrn', 'eventContent', 'subtype', 'from', 'previousEventInConversation', 'originToken', 'backendUrn'])

In [30]:
conv.keys()

dict_keys(['metadata', 'elements', 'paging'])

In [34]:
conv['elements'][0]

{'dashEntityUrn': 'urn:li:fsd_conversation:2-MzVmYTJhOWEtZWFjYy00NTQ0LWI5NWQtOWMxNDFlOTk4ZDI5XzAxMg==',
 'inboxType': 'PRIMARY',
 'notificationStatus': 'ACTIVE',
 'read': True,
 'groupChat': False,
 'totalEventCount': 1,
 'unreadCount': 0,
 'lastActivityAt': 1685453603005,
 'firstMessageUrn': 'urn:li:fs_event:(2-MzVmYTJhOWEtZWFjYy00NTQ0LWI5NWQtOWMxNDFlOTk4ZDI5XzAxMg==,2-MTY4NTM5NDc0NTA0MmI5OTUxMi0wMDMmMzVmYTJhOWEtZWFjYy00NTQ0LWI5NWQtOWMxNDFlOTk4ZDI5XzAxMg==)',
 'backendUrn': 'urn:li:messagingThread:2-MzVmYTJhOWEtZWFjYy00NTQ0LWI5NWQtOWMxNDFlOTk4ZDI5XzAxMg==',
 'receipts': [],
 'lastReadAt': 1685394766801,
 'archived': False,
 'blocked': False,
 'starred': False,
 'entityUrn': 'urn:li:fs_conversation:2-MzVmYTJhOWEtZWFjYy00NTQ0LWI5NWQtOWMxNDFlOTk4ZDI5XzAxMg==',
 'sdkEntityUrn': 'urn:li:msg_conversation:(urn:li:fsd_profile:ACoAADRK5hYBGMJvZMHyTclwGIHUeJVuGK-eN_A,2-MzVmYTJhOWEtZWFjYy00NTQ0LWI5NWQtOWMxNDFlOTk4ZDI5XzAxMg==)',
 'viewerCurrentParticipant': True,
 'featureTypes': ['REACTIONS',
 

In [48]:
t=test_api.get_profile('jesus-imbotte-871850273')

In [49]:
t

{'industryName': 'Internet',
 'lastName': 'Imbotte',
 'locationName': 'France',
 'student': False,
 'geoCountryName': 'France',
 'geoCountryUrn': 'urn:li:fs_geo:105015875',
 'geoLocationBackfilled': False,
 'elt': True,
 'industryUrn': 'urn:li:fs_industry:6',
 'firstName': 'Jesus',
 'entityUrn': 'urn:li:fs_profile:ACoAAELd6dUBqY8EezTdxSClo5LBDgLVpTr_aAM',
 'geoLocation': {'geoUrn': 'urn:li:fs_geo:101240143', 'postalCode': '75001'},
 'geoLocationName': 'Paris, Île-de-France',
 'location': {'basicLocation': {'countryCode': 'fr', 'postalCode': '75001'}},
 'headline': 'Scientifique des données chez Google',
 'profile_id': 'ACoAAELd6dUBqY8EezTdxSClo5LBDgLVpTr_aAM',
 'profile_urn': 'urn:li:fs_miniProfile:ACoAAELd6dUBqY8EezTdxSClo5LBDgLVpTr_aAM',
 'member_urn': 'urn:li:member:1121839573',
 'public_id': 'jesus-imbotte-871850273',
 'experience': [{'entityUrn': 'urn:li:fs_position:(ACoAAELd6dUBqY8EezTdxSClo5LBDgLVpTr_aAM,2164463015)',
   'companyName': 'Google',
   'company': {'employeeCountRang

In [40]:
t['entityUrn'].split(":")[-1]

'ACoAAELd6dUBqY8EezTdxSClo5LBDgLVpTr_aAM'

In [50]:
real_urn = test_api.get_profile('tanguy-bellec-646992205')['entityUrn'].split(":")[-1]

In [51]:
real_urn

'ACoAADRK5hYBGMJvZMHyTclwGIHUeJVuGK-eN_A'

In [53]:
test_api.get_profile_connections(real_urn)

[]

In [55]:
real_api.get_profile_connections(real_urn)

[]

In [37]:
print('ihger')

ihger


In [46]:
con = test_api.get_profile_connections(test_api.get_profile('tanguy-bellec-646992205')['entityUrn'].split(":")[-1])

In [47]:
con

[]

In [ ]:
con = real_api.get_profile_connections(t['entityUrn'].split(":")[-1])

In [45]:
con

[]

# Work to get optimzed messages

In [11]:
import openai

In [32]:
openai.api_key = open_file('open_ai_key.txt')[2:-2]

In [ ]:
mess_quant = "graduate from an MSc in Statistics and Quantitative Finance. An experience as a data scientist, looking for an opportunity as a quant"
mess_data = "graduate from an MSc in Statistics. An experience as a data scientist, looking for a new opportunity, data-driven."

In [13]:
def gpt3_chat(message: str, messages:list[dict])-> tuple[str, list]:
    '''Returns GPT's response to the input message and appends it to the messages list'''
    messages.append({"role": "user", "content": message})
    # ChatGPT is powered by gpt-3.5-turbo, OpenAI’s most advanced language model.
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)['choices'][0]['message']['content']
    messages.append({"role": "assistant", "content": response})
    return str(response), messages

In [14]:
def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

test = test_api.get_profile(search_quant[7]['public_id'])
test['summary'][0:500]

In [15]:
discussion = [
        {"role": "system", "content": "You are a helpful assistant that personalize generic message asking for a connection on linkedin to headhunters from their summary in no more than 300 character."},
        {"role": "user", "content": f"Here is the generic message : <<{mess}>>"},
        {"role": "assistant", "content": "What is the summary from the headhunter?"},
        {"role": "user", "content": f"Here is the summary from his profile : <<{test['summary']}>>"},
        {"role": "assistant", "content": "What is the expected output format?"},        
]


message_prompt = "No more than 240 character. Just the motivation part, no intro or outro. Don't say hello. Do it now."
answer, discussion = gpt3_chat(message=message_prompt, messages=discussion)
print(answer)
answers.append([answer, discussion])

NameError: name 'mess' is not defined

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant that personalize generic message asking for a connection on linkedin to headhunters from their summary."},
        {"role": "user", "content": f"Here is the generic message : <<{mess}>>"},
        {"role": "assistant", "content": "What is the summary from the headhunter?"},
        {"role": "user", "content": f"Here is the summary from his profile : <<{prof['summary']}>>"},
        {"role": "assistant", "content": "What is the expected output format?"},        
    ]
)

experience_prompt = "A motivation message in no more than 50 words to invite the headhunter to contact me if he sees a fit. Do it now."
experience, cv_discussion = utils.gpt3_chat(message=experience_prompt, messages=cv_discussion)
print(experience)

# Work to find optimal job posts

- When an interesting job is found, we want to connect with the recruiter but we can't get his profile from the get_job() func already implemented
- We will try to add a func to the class linkedin to get this data

In [209]:
test_job = test_api.test_get_job(jobs[0]['entityUrn'].split(":")[3])

in test_job


In [4]:
test = test_api.search_jobs(keywords = 'Quant quantitative researcher',
                                limit = 100, listed_at = 14 * 24 * 60 * 60, location_name = 'London, England')

In [5]:
test

[{'dashEntityUrn': 'urn:li:fsd_jobPosting:3642500699',
  'companyDetails': {'company': 'urn:li:fs_normalized_company:14838829',
   '*companyResolutionResult': 'urn:li:fs_normalized_company:14838829',
   '$recipeTypes': ['com.linkedin.voyager.deco.jserp.WebJobPostingWithCompanyName'],
   '$type': 'com.linkedin.voyager.jobs.JobPostingCompany'},
  'formattedLocation': 'London, England, United Kingdom',
  'listedAt': 1687482564000,
  'title': 'Credit Quantitative Researcher VP',
  '$recipeTypes': ['com.linkedin.voyager.deco.jserp.WebSearchJobJserpJobPostingLite'],
  '$type': 'com.linkedin.voyager.jobs.JobPosting',
  'entityUrn': 'urn:li:fs_normalized_jobPosting:3642500699',
  'workRemoteAllowed': False},
 {'dashEntityUrn': 'urn:li:fsd_jobPosting:3649926118',
  'companyDetails': {'company': 'urn:li:fs_normalized_company:35445621',
   '*companyResolutionResult': 'urn:li:fs_normalized_company:35445621',
   '$recipeTypes': ['com.linkedin.voyager.deco.jserp.WebJobPostingWithCompanyName'],
   '$

In [17]:
test = test_api.search_jobs(keywords = 'Data Scientist ML AI', title = 'Data Scientist',
                                limit = 100, listed_at = 14 * 24 * 60 * 60, location_name = 'London, England')

In [21]:
DS_UK_jobs = test_api.search_jobs(keywords = 'Data Scientist', title = 'Data Scientist',
                                limit = 100, listed_at = 14 * 24 * 60 * 60, location_name = 'London, England')


In [8]:
test = test_api.search_jobs(keywords = 'Data Scientist ML AI', title = 'Data Scientist',
                                limit = 100, listed_at = 14 * 24 * 60 * 60, location_name = 'Paris, Île-de-France, France')

In [6]:
test_job = test_api.search_jobs(keywords = 'quantitative researcher', experience = "2", 
                                limit = 100, listed_at = 1 * 24 * 60 * 60)

In [170]:
jobs = test_api.search_jobs(keywords = 'data', limit = 5)

# Work with the package selenium to fetch data from recruiter

In [22]:
import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import unittest


In [23]:
def connect_firefox_to_my_linkedin(credentials):
    
    browser = webdriver.Firefox()

    # your secret credentials:
    email = credentials[0]
    password = credentials[1]
    # Go to linkedin and login
    browser.get('https://www.linkedin.com/login')
    time.sleep(3)
    browser.find_element(by =By.ID, value ='username').send_keys(email)
    browser.find_element(by =By.ID, value ='password').send_keys(password)
    browser.find_element(by =By.ID, value ='password').send_keys(Keys.RETURN)
    
    time.sleep(3)
    
    return browser


def get_recruiter_id_from_job(browser, job_id):
    
    browser.get(f"https://www.linkedin.com/jobs/view/{job_id}")

    time.sleep(2)
    
    sele_search = browser.find_elements(by=By.CLASS_NAME, value="mh4.pt4.pb3")
    
    if(len(sele_search) == 0):
        return False
    
    recruiter_public_id = sele_search[0].find_element(by=By.CSS_SELECTOR,value ='a').get_attribute('href').split('/')[-1]
    
    return recruiter_public_id

In [24]:
browser = connect_firefox_to_my_linkedin(test_credential)
get_id = get_recruiter_id_from_job(browser, "3583408428")

# Work to link our message to a collab sheet

https://towardsdatascience.com/turn-google-sheets-into-your-own-database-with-python-4aa0b4360ce7

In [25]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# attributes that can be of use :
#    - sheet.get_all_values()
#    - sheet.row_values(n) n:int

In [49]:
def add_new_row(sheet, row_to_insert):
    
    current_number_rows = len(sheet.get_all_values())
    sheet.insert_row(row_to_insert, current_number_rows + 1)

    return True

def delete_last_row(sheet):
    
    current_number_rows = len(sheet.get_all_values())
    sheet.delete_rows(current_number_rows)
    
    return True


def create_sheet(sheet_name, mail):
    
    scope = ['https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive"]

    credentials = ServiceAccountCredentials.from_json_keyfile_name("gs_credentials.json", scope)
    client = gspread.authorize(credentials)

    sheet = client.create(sheet_name)
    sheet.share(mail, perm_type='user', role='writer')
    
    return sheet


def connect_to_sheet(sheet_name):
    
    scope = ['https://www.googleapis.com/auth/spreadsheets',
            "https://www.googleapis.com/auth/drive"]

    credentials = ServiceAccountCredentials.from_json_keyfile_name("gs_credentials.json", scope)
    client = gspread.authorize(credentials)
    
    sheet = client.open(sheet_name).sheet1
    
    return sheet

In [33]:
### Only create the sheet once, if it has been deleted, use this command

#mail = "tang.bellec@gmail.com"
#sheet = create_sheet("messages_sent", mail)
#sheet = connect_to_sheet(sheet_name)
#add_new_row(sheet, ["job_id", "recruiter_id", "message"])

True

In [50]:
sheet = connect_to_sheet("messages_sent")

In [51]:
add_new_row(sheet, ["job_id", "recruiter_id", "message"])

True

# Pipeline to dm recruiters from job post

https://www.linkedin.com/mynetwork/invitation-manager/sent/ link to see invite sent

### extract job title

In [27]:
import json

def json_to_dict(json_string):
    """Converts a JSON-formatted string to a Python dictionary."""
    return json.loads(json_string)

def gpt_extract_job_function_from_title(job_title):

    prompt = f"""
    Your task is to extract the job title from a job post title. \
    We just want to keep the job function, no more than 3 words.
    Provide it in JSON format with the key: job_title 

    Job post title: ```{job_title}```
    """

    response = get_completion(prompt)
    response = json_to_dict(response)
    
    return response


def add_easy_connection_from_recruiter_job_post(recruiter_id, api):
    
    message = ""
    
    current_recruiter = api.get_profile(recruiter_id)

    first_name = get_first_name_fetched_profile(current_recruiter)
    print(first_name)
    
    is_french = is_profile_fetched_french(current_recruiter)
    print(is_french)
    
    sector = is_profile_data_or_finance(current_recruiter)
    print(sector)

    if(is_french):
        print("french")
        
    else:
        
        if(sector == 'finance'):
            message = "Hello " + first_name + "\n" + message_quant + "\n" + end_of_message

        else:
            message = "Hello " + first_name + "\n" + message_data + "\n" + end_of_message


    connection = api.add_connection(current_recruiter['public_id'], message)
    
    return [sector, connection]


def create_message_for_recruiter_from_job(recruiter_profile_id, job_title, api = test_api):
    
    recruiter = api.get_profile(recruiter_profile_id)
    first_name = recruiter['firstName']
    
    if(first_name):
        
        message = "Hello "+ first_name + ", \n" + "I am a graduate from an Msc in Statistics with an experience" \
                  + "as a Data Scientist and I came across your job post for a " + job_title \
                  + " role. I am very interested in this opportunity and would love to learn more about the position."

    else:
        
        message = "Hello, I am a graduate from an Msc in Statistics with an experience" \
                  + "as a Data Scientist and I came across your job post for a " + job_title \
                  + " role. I am very interested in this opportunity and would love to learn more about the position."

    
    return message

In [54]:
DS_UK_jobs = test_api.search_jobs(keywords = 'Data Scientist', title = 'Data Scientist',
                                limit = 100, listed_at = 7 * 24 * 60 * 60, location_name = 'London, England')



In [46]:
test = test_api.search_jobs(keywords = 'Machine Learning',
                                limit = 1000, listed_at = 7 * 24 * 60 * 60, location_name = 'London, England')

In [57]:
browser = connect_firefox_to_my_linkedin(true_credential)

In [61]:
#browser = connect_firefox_to_my_linkedin(true_credential)

no_id = 0
jobs = test

for job in jobs[300:400]:
    
    current_job_id = job['entityUrn'].split(":")[3]
    recruiter_id = get_recruiter_id_from_job(browser, current_job_id)
    
    if(recruiter_id):
        
        job_title = job['title']
        job_title = gpt_extract_job_function_from_title(job_title)['job_title']
        
        message = create_message_for_recruiter_from_job(recruiter_id, job_title)
        
        connection = real_api.add_connection(recruiter_id, message)
        
        add_new_row(sheet, [current_job_id, recruiter_id, message])
    
        print(message)
    else:
        print("can't get id")
        no_id += 1
        
    time.sleep(3)
    
    
    
        

can't get id
Hello Augusta, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Quantitative Researcher role. I am very interested in this opportunity and would love to learn more about the position.
can't get id
Hello Paul, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Artificial Intelligence Engineer role. I am very interested in this opportunity and would love to learn more about the position.
can't get id
can't get id
can't get id
Hello Aliur, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Analytics Consultant role. I am very interested in this opportunity and would love to learn more about the position.
Hello Jesse, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Quantitative Researcher role. I am very 

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

# Finding jobs and easy apply to them

In [4]:
jobs = test_api.search_jobs(
        keywords='data scientist',
        companies=None,
        experience=None,
        job_type=None,
        job_title=None,
        industries=None,
        location_name='Paris, Île-de-France, France',
        remote=False,
        easyApply = True,
        listed_at=24 * 60 * 60,
        distance=None,
        limit=10,
        offset=0
    )

In [6]:
jobs[0]

{'dashEntityUrn': 'urn:li:fsd_jobPosting:3648714460',
 'companyDetails': {'company': 'urn:li:fs_normalized_company:3234841',
  '*companyResolutionResult': 'urn:li:fs_normalized_company:3234841',
  '$recipeTypes': ['com.linkedin.voyager.deco.jserp.WebJobPostingWithCompanyName'],
  '$type': 'com.linkedin.voyager.jobs.JobPostingCompany'},
 'formattedLocation': 'Île-de-France, France',
 'listedAt': 1687866988000,
 'title': 'Biostatisticien/Biostatisticienne',
 '$recipeTypes': ['com.linkedin.voyager.deco.jserp.WebSearchJobJserpJobPostingLite'],
 '$type': 'com.linkedin.voyager.jobs.JobPosting',
 'entityUrn': 'urn:li:fs_normalized_jobPosting:3648714460',
 'workRemoteAllowed': False}